# Deploy Web App on Azure Container Services (AKS)


In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending an image to it and getting it scored. 

The process is split into the following steps:
- Define our resource names
- Login to Azure
- Create resource group and create AKS
- Connect to AKS
- Deploy our app
- Tear it all down

We assume that this notebook is running on Linux and Azure CLI is installed before proceeding.

In [1]:
import json
from testing_utilities import write_json_to_file

## Setup

Below are the various name definitions for the resources needed to setup AKS as well as the name of the Docker image we will be using. If you wish to use the image that you previously pushed to your account, make sure you change this.

In [2]:
# Please modify the below as you see fit
resource_group = "<RESOURCE_GROUP>" 
aks_name = "<AKS_CLUSTER_NAME>"
location = "eastus"

docker_login = '<YOUR_DOCKER_LOGIN>'
image_name = docker_login + '/kerastf-gpu' # 'fboylu/kerastf-gpu' Feel free to use this image if you want to 
                                           # skip creating your own container
selected_subscription = "'<YOUR_SUBSCRIPTION>'" # If you have multiple subscriptions select 
                                                # the subscription you want to use here

## Azure account login

The command below will initiate a login to your Azure account. It will pop up with an url to go to where you will enter a one off code and log into your Azure account using your browser.

In [ ]:
!az login -o table

In [ ]:
!az account set --subscription $selected_subscription

In [ ]:
!az account show

You will also need to register the container service resources on your subscription if you haven't already done so.

In [ ]:
!az provider register -n Microsoft.ContainerService

In [ ]:
!az provider show -n Microsoft.ContainerService

## Create resources and dependencies

### Create resource group and AKS cluster

Azure encourages the use of groups to organize all the Azure components you deploy. That way it is easier to find them but also we can delete a number of resources simply by deleting the group.

In [ ]:
!az group create --name $resource_group --location $location

Below, we create the AKS cluster in the resource group we created earlier. This could take up to 15 minutes.

In [8]:
%%time
!az aks create --resource-group $resource_group --name $aks_name --node-count 1 --generate-ssh-keys -s Standard_NC6

 - Finished ..{
  "aadProfile": null,
  "addonProfiles": null,
  "agentPoolProfiles": [
    {
      "count": 1,
      "dnsPrefix": null,
      "fqdn": null,
      "maxPods": 110,
      "name": "nodepool1",
      "osDiskSizeGb": null,
      "osType": "Linux",
      "ports": null,
      "storageProfile": "ManagedDisks",
      "vmSize": "Standard_NC6",
      "vnetSubnetId": null
    }
  ],
  "dnsPrefix": "fbAKSClust-fbaksrg-e984a9",
  "enableRbac": true,
  "fqdn": "fbaksclust-fbaksrg-e984a9-bf6af1df.hcp.eastus.azmk8s.io",
  "id": "/subscriptions/XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX/resourcegroups/fbaksrg/providers/Microsoft.ContainerService/managedClusters/fbAKSCluster",
  "kubernetesVersion": "1.9.9",
  "linuxProfile": {
    "adminUsername": "azureuser",
    "ssh": {
      "publicKeys": [
        {
          "keyData": "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDA912ocZhmAUJbpPW/mOg5anphH0ehCLNlgofEvuCCwIV/2oRBL2uC6F2fDImkOjwSLQwOdhK+IvLOBUTdBOSXwOG+Dv6kd0n+YIKrykI60UJJjx+gYw3lFqeSgkjavKQpcQ

### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [9]:
!sudo env "PATH=$PATH" az aks install-cli

Please ensure that /usr/local/bin is in your search PATH, so the `kubectl` command can be found.


## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [ ]:
!az aks get-credentials --resource-group $resource_group --name $aks_name

Let's verify connection by listing the nodes.

In [12]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-28016997-0   Ready     agent     55d       v1.9.6


Let's check the pods on our cluster.

In [13]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
kube-system   azureproxy-79c5db744-r5ggd              1/1       Running   2          55d
kube-system   heapster-55f855b47-4m7xr                2/2       Running   0          55d
kube-system   kube-dns-v20-7c556f89c5-4z4z6           3/3       Running   0          55d
kube-system   kube-dns-v20-7c556f89c5-mp5fh           3/3       Running   0          55d
kube-system   kube-proxy-k8t2c                        1/1       Running   0          55d
kube-system   kube-svc-redirect-z6ppp                 1/1       Running   8          55d
kube-system   kubernetes-dashboard-546f987686-8krxm   1/1       Running   2          55d
kube-system   tunnelfront-695bcbdc68-t4l8t            1/1       Running   27         55d


## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the volume mounts to the drivers that are located on the node.

In [14]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "replicas":1,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-dl"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-dl",
                      "image": image_name,
                      "env":[
                          {
                              "name": "LD_LIBRARY_PATH",
                              "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.5/lib"
                          }
                      ],
                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "volumeMounts":[
                          {
                              "mountPath":"/usr/local/nvidia",
                              "name": "nvidia",
                          }
                      ],
                      "resources":{
                           "requests":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           },
                           "limits":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           }
                       }  
                  }
              ],
              "volumes":[
                  {
                      "name": "nvidia",
                      "hostPath":{
                          "path":"/usr/local/nvidia"
                      },
                  },
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-dl"
      }
   }
}

In [15]:
write_json_to_file(app_template, 'az-dl.json')

In [16]:
write_json_to_file(service_temp, 'az-dl.json', mode='a')

Let's check the manifest created.

In [17]:
!cat az-dl.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-dl"
    },
    "spec": {
        "replicas": 1,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-dl"
                }
            },
            "spec": {
                "containers": [
                    {
                        "env": [
                            {
                                "name": "LD_LIBRARY_PATH",
                                "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.5/lib"
                            }
                        ],
                        "image": "fboylu/kerastf-gpu",
                        "name": "azure-dl",
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
               

Next, we will use kubectl create command to deploy our application.

In [18]:
!kubectl create -f az-dl.json

deployment.apps/azure-dl created
service/azure-dl created


Let's check if the pod is deployed. It may take as many as 10 minutes for the container to be ready.

In [21]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-dl-b47cf8cdd-4gg6v                1/1       Running   0          3m
kube-system   azureproxy-79c5db744-r5ggd              1/1       Running   2          55d
kube-system   heapster-55f855b47-4m7xr                2/2       Running   0          55d
kube-system   kube-dns-v20-7c556f89c5-4z4z6           3/3       Running   0          55d
kube-system   kube-dns-v20-7c556f89c5-mp5fh           3/3       Running   0          55d
kube-system   kube-proxy-k8t2c                        1/1       Running   0          55d
kube-system   kube-svc-redirect-z6ppp                 1/1       Running   8          55d
kube-system   kubernetes-dashboard-546f987686-8krxm   1/1       Running   2          55d
kube-system   tunnelfront-695bcbdc68-t4l8t            1/1       Running   27         55d


If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [22]:
!kubectl get events

LAST SEEN   FIRST SEEN   COUNT     NAME                                         KIND         SUBOBJECT                   TYPE      REASON                  SOURCE                              MESSAGE
25m         25m          1         azure-dl-66f69c4f79-mpmvh.15471bba8ebb833f   Pod          spec.containers{azure-dl}   Normal    Killing                 kubelet, aks-nodepool1-28016997-0   Killing container with id docker://azure-dl:Need to kill Pod
3m          3m           1         azure-dl-b47cf8cdd-4gg6v.15471ceb30164ee8    Pod                                      Normal    Scheduled               default-scheduler                   Successfully assigned azure-dl-b47cf8cdd-4gg6v to aks-nodepool1-28016997-0
3m          3m           1         azure-dl-b47cf8cdd-4gg6v.15471ceb3ec741e6    Pod                                      Normal    SuccessfulMountVolume   kubelet, aks-nodepool1-28016997-0   MountVolume.SetUp succeeded for volume "nvidia" 
3m          3m           1         azure-dl

Check the logs for the application pod.

In [26]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))

In [27]:
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

2018-08-02 16:09:17,655 CRIT Supervisor running as root (no user in config file)
2018-08-02 16:09:17,658 INFO supervisord started with pid 1
2018-08-02 16:09:18,660 INFO spawned: 'program_exit' with pid 9
2018-08-02 16:09:18,662 INFO spawned: 'nginx' with pid 10
2018-08-02 16:09:18,664 INFO spawned: 'gunicorn' with pid 11
2018-08-02 16:09:19,695 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-08-02 16:09:20.328936: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-08-02 16:09:20.512472: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1392] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: ddde:00:00.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-08-02 16:09:20.512518: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1471] Adding visible gpu devices: 0
2018-08-02 

In [28]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-dl   1         1         1            1           4m


It can take a few minutes for the service to populate the EXTERNAL-IP field below. This will be the IP you use to call the service. You can also specify an IP to use, please see the AKS documentation for further details.

In [29]:
!kubectl get service azure-dl

NAME       TYPE           CLUSTER-IP   EXTERNAL-IP    PORT(S)        AGE
azure-dl   LoadBalancer   10.0.9.65    40.87.54.254   80:30390/TCP   4m


Next, we will [test our web application deployed on AKS](05_TestWebApp.ipynb). Once, we are done with all the notebooks of the tutorial, below instructions can be used to delete the cluster and free resources.

# Tear it all down

Once you are done with your cluster you can use the following two commands to destroy it all.

In [42]:
!kubectl delete -f az-dl.json

deployment.apps "azure-dl" deleted
service "azure-dl" deleted


In [43]:
!az aks delete -n $aks_name -g $resource_group -y

inished ..

In [44]:
!az group delete --name $resource_group -y

inished ..